In [30]:
pip install pyautogui

In [31]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install pyzbar

Note: you may need to restart the kernel to use updated packages.


In [34]:

#AQUI SE MANDAN A IMPORTAR LAS LIBRERIAS 

import cv2
import openpyxl
import qrcode
import os
import tkinter as tk
from tkinter import ttk
from tkinter import Label
import pandas as pd
from pyzbar.pyzbar import decode  # Para decodificar los códigos QR
import sqlite3

In [35]:
print("OpenCV Version:", cv2.__version__, openpyxl.__version__)

OpenCV Version: 4.7.0 3.1.2


In [36]:
# RUTA DEL DIRECTORIA DE NUESTRA BASE DE DATOS EN EXCEL 
bd_excel = r"C:\Users\suaba\Desktop\PROYECTOFINALQR/proyectofinalqr.xlsx"

# NOMBRE DE HOJA DE NUESTRO EXCEL
nombre_hoja = "registros"

# CARPETA DONDE SE GUARDARAN LOS CODIGOS QR
directorio_destino = r"C:\Users\suaba\Desktop\PROYECTOFINALQR\codigosqr"

# ABRIR EXCEL 
workbook = openpyxl.load_workbook(bd_excel)

# SELECCION DE HOJA
hoja = workbook[nombre_hoja]
oracion = f"Se está trabajando con el archivo '{bd_excel}' en la hoja '{nombre_hoja}' del documento./ '{hoja} '"
print(oracion)

Se está trabajando con el archivo 'C:\Users\suaba\Desktop\PROYECTOFINALQR/proyectofinalqr.xlsx' en la hoja 'registros' del documento./ '<Worksheet "registros"> '


In [37]:

# SOLO SE CORRE PARA GENERAR EL QR DE EL REGISTRO QUE SE NECESITA

valor_buscado = int(input("ingrese el dato de busqueda"))

#CARGAR EXCEL
excel_file = 'proyectofinalqr.xlsx'
df = pd.read_excel(excel_file, engine='openpyxl')

#CONEXION BD

conn = sqlite3.connect('RegistroBD.db')

# Agrega los datos del DataFrame a la tabla de la base de datos
df.to_sql('Usuarios', conn, if_exists='replace', index=False)


print(df)
resultado = df.loc[df['id_clave'] == valor_buscado, 'id_clave']

# Verifica si se encontró el valor y muestra el resultado

valor_en_columna_F = resultado.values[0]
print(f'El valor numero de id_clave de "{valor_buscado}" es: {valor_en_columna_F}')

   id_clave  Nombre Apellido    Padre      Numero Codigo QR
0   1926598   ERIK    MENDEZ  ANTONIO  5618503856     RUTA1
1   1973155     ALE   SUAREZ     JOSE  5613138133     RUTA2
2   2126812     FER   SUAREZ      ALE  5626680186     RUTA3
3   1926486  JIMENA  BASILIO     OMAR  5583233548     RUTA4
El valor numero de id_clave de "1926598" es: 1926598


In [38]:

# BUSCA EN LA COLUMNA DE ID_CLAVE Y ME DA EL NUMERO DE TELEFONO(el Folio)
# USANDO DF

#CARGAR BD EXCEL
excel_file = 'proyectofinalqr.xlsx'
df = pd.read_excel(excel_file, engine='openpyxl')

# CONEXION BD
conn = sqlite3.connect('RegistroBD.db')

df.to_sql('Usuarios', conn, if_exists='replace', index=False)

# Muestra el DataFrame con todos los datos
print(df)

# Busca la fila en la que el valor de 'Column2' sea igual a 'B'
fila_seleccionada = df.loc[df['id_clave'] == valor_buscado]

if not fila_seleccionada.empty:
    columna_folio = fila_seleccionada['id_clave'].values[0]
    print(f"Fila seleccionada: \n{fila_seleccionada}")
    print(f"Valor de la columna 0: {columna_folio}")
else:
    print("No se encontraron filas con el valor 'B' en 'Column2'")

#CIERRE CONEXION BD
conn.close()

   id_clave  Nombre Apellido    Padre      Numero Codigo QR
0   1926598   ERIK    MENDEZ  ANTONIO  5618503856     RUTA1
1   1973155     ALE   SUAREZ     JOSE  5613138133     RUTA2
2   2126812     FER   SUAREZ      ALE  5626680186     RUTA3
3   1926486  JIMENA  BASILIO     OMAR  5583233548     RUTA4
Fila seleccionada: 
   id_clave Nombre Apellido    Padre      Numero Codigo QR
0   1926598  ERIK    MENDEZ  ANTONIO  5618503856     RUTA1
Valor de la columna 0: 1926598


In [39]:
columna_folio=str(columna_folio)

In [40]:
#CREACION DE CODIGO QR
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)

# DATOS FOLIO
qr.add_data(columna_folio)
qr.make(fit=True)

#IMAGEN QR
img = qr.make_image(fill_color="black", back_color="white")

import os

# DIRECTORIO IMGENES QR
directorio_destino = r"C:\Users\suaba\Desktop\PROYECTOFINALQR\codigosqr"

#GUARDAR IMAGEN QR EN CARPETA ESPECIFICA
nombre_archivo_qr = f"codigo_qr_{columna_folio}.png"

# Eliminar espacios en blanco al final del nombre del archivo
nombre_archivo_qr = nombre_archivo_qr.strip()

ruta_completa = os.path.join(directorio_destino, nombre_archivo_qr)

img.save(ruta_completa)
img.show()

In [41]:

#LECTURA DE CAMARA PARA LEER EL CODIGO QR
import cv2 
import numpy as np 
from pyzbar.pyzbar import decode

cap= cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    
    if not success:
        break
    
    for code in decode(img):
        
        decoded_data = code.data.decode("utf-8")
        
        print(decoded_data)
        
        rect_pts = code.rect
        
        if decoded_data:
            pts = np.array([code.polygon], np.int32)
            cv2.polylines(img, [pts], True,(255,0,0),3)
            cv2.putText(img, str(decoded_data),(rect_pts[0], rect_pts[1]),cv2.FONT_HERSHEY_COMPLEX_SMALL,1 ,(0,255, 0),)

        # CIERRE DE CAMARA
        cap.release()

        #VENTANA PARA EL CODIGO QR
        cv2.imshow("QR Code Content", img)
        cv2.waitKey(0)  #PRESIONA CUALQUIER TECLA
decoded_data = int(decoded_data)
#CIERRA LAS VENTANAS
cv2.destroyAllWindows()
# PROCEDE A LEER EL DATO Y LO QUE VA A BUSCAR ES UN NUMERO DE TELEFONO
# BUSCA EN LA COLUMNA DE ID_CLAVE Y ME DA EL NUMERO DE TELEFONO
# USANDO DF
import pandas as pd
import sqlite3
#CARGAR NUESTRA BD DE EXCEL
excel_file = 'proyectofinalqr.xlsx'
df = pd.read_excel(excel_file, engine='openpyxl')

#CONEXION BD
conn = sqlite3.connect('RegistroBD.db')

# Agrega los datos del DataFrame a la tabla de la base de datos
df.to_sql('Usuarios', conn, if_exists='replace', index=False)

fila_seleccionada = df.loc[df['id_clave'] == decoded_data]

if not fila_seleccionada.empty:
    valor_columna5 = fila_seleccionada['Numero'].values[0]
    print(f"Fila seleccionada: \n{fila_seleccionada}")
    print(f"Valor de la columna 5: {valor_columna5}")
else:
    print("No se encontraron filas con el valor 'B' en 'Column2'")

#CONEXION BD CERRADA
conn.close()

valor_columna5 = str(valor_columna5)


#LIBRERIAS
import pywhatkit as kit
from datetime import datetime
import time
import pyautogui as pg

#FECHA
fecha_hora_actual = datetime.now()

#SE EXTRAE LA FECHA,HORA Y MINUTOS
fecha_actual = fecha_hora_actual.strftime("%Y-%m-%d")
hora_actual = fecha_hora_actual.strftime("%H:%M")
minutos_actual = fecha_hora_actual.strftime("%M")

# IMPRIME LA FECHA, HORA Y MINUTOS
print("Fecha actual:", fecha_actual)
print("Hora actual:", hora_actual)
print("Minutos actuales:", minutos_actual)



phone_number = str("+52" + valor_columna5)
phone_number

message = "USUARIO REGISTRADO EN TECNOLOGIAS COMPUTACIONALES II PARA PROYECTO FINAL  " + hora_actual + " hoy : " + fecha_actual
second = time.time()+60
date = datetime.fromtimestamp(second)

#for i in range(35):
kit.sendwhatmsg(phone_number,message, date.hour, date.minute)
pg.hotkey('enter')

time.sleep(30)
pg.hotkey('ctrl','w')
#PROYECTO FINALIZADO

1973155
Fila seleccionada: 
   id_clave Nombre Apellido Padre      Numero Codigo QR
1   1973155    ALE   SUAREZ  JOSE  5613138133     RUTA2
Valor de la columna 5: 5613138133
Fecha actual: 2023-11-09
Hora actual: 16:36
Minutos actuales: 36
In 19 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
